In [14]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from tqdm import tqdm
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_labels
import random

random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    if mask_rle == -1:
        return np.zeros(shape, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    if np.sum(mask) == 0:
        return '-1'
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 홀 채우기 함수 수정
def mask_filling(mask):
    # 마스크 내부 영역 추출
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_mask = np.zeros_like(mask)
    cv2.drawContours(filled_mask, contours, -1, 255, -1)
    return filled_mask

# CSV 파일 로드
csv_file = pd.read_csv('./result/b5_vanila.csv')  # CSV 파일 경로

# 결과를 저장할 새로운 DataFrame 생성
result_data = pd.DataFrame(columns=['img_id', 'mask_rle'])

# 각 사진과 마스크에 대해 예시 출력 및 결과 저장
for i in tqdm(range(9)):
    # 사진 파일 경로 생성
    
    image_path = f'./split_custom/swbio_img/{csv_file["img_id"][i]}.png'  # 이미지 파일 경로 생성 (확장자에 따라 수정해야 할 수 있습니다)

    # 사진 파일 확인
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found: {image_path}")
    except Exception as e:
        print(str(e))
        assert False

    # 마스크 정보 로드 및 디코딩
    mask_data = csv_file['mask_rle'][i]
    mask = rle_decode(mask_data, image.shape[:2])

    # 작은 객체 제거
    min_object_size = 10  # 임의의 작은 객체 크기 기준 설정
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_object_size:
            cv2.drawContours(mask, [contour], 0, 0, -1)

    # 마스크 내부 영역 채우기
    filled_mask = mask_filling(mask)

    # DenseCRF2D 입력 설정

    ############## 재욱이 추가 부분 ##################
    # unary = np.expand_dims(mask, axis=0)
    # unary = np.concatenate([unary, 1-unary], axis=0)
    # unary = unary.reshape((2, -1))
    # unary_float = unary.astype(np.float32)

    # # DenseCRF2D 객체 생성
    # d = dcrf.DenseCRF2D(224, 224, 2)  # width, height, nlabels

    # # unary potential 설정
    # d.setUnaryEnergy(unary_float)

    # d.addPairwiseGaussian(sxy=3, compat=3)

    # # DenseCRF2D 수행
    # Q = d.inference(10)  # 반복 횟수 설정

    # # 최적화된 마스크 가져오기
    # filled_mask = np.argmax(Q, axis=0).reshape((224, 224))


    # 결과를 DataFrame에 저장
    result_data.loc[i] = [csv_file['img_id'][i], rle_encode(filled_mask)]

# 결과 DataFrame을 final.csv 파일로 저장
result_data['mask_rle'] = result_data['mask_rle'].fillna(-1)
print(result_data.head)


100%|██████████| 9/9 [00:00<00:00, 11.17it/s]

<bound method NDFrame.head of   img_id                                           mask_rle
0      0  20102 5 20151 10 20325 8 20374 11 20549 8 2059...
1      1  20833 2 21057 5 21281 5 21505 6 21729 6 21953 ...
2      2                                                 -1
3      3  213 6 437 6 662 5 886 3 33790 4 34013 6 34236 ...
4      4  10522 2 10745 3 10967 6 11190 7 11413 8 11636 ...
5      5  10977 2 11201 3 11425 3 11649 4 11873 6 12097 ...
6      6  45921 2 46145 3 46369 3 46593 3 46817 3 47041 ...
7      7                                                 -1
8      8                                                 -1>


In [13]:
result_data.to_csv('./result/b5_vanila_yjypost.csv', index=False)